In [1]:
import pandas as pd

In [5]:
### EXTRACT THE PRODUCTION RELATION DATA
df_prod = pd.read_excel('./sources/production_data.xlsx', sheet_name='Data')
df_code = pd.read_excel('./sources/production_codes.xlsx', sheet_name='Data')

# filter out the rows of data not applicable to project
valid_codes = df_code['msnCode'].tolist()
df_prod = df_prod.loc[df_prod['MSN'].isin(valid_codes)]

# list of lists that will later become final dataframe
data = []

# convert each year column of energy produced into individual rows:
for i in range(3, 62):
    
    # get values from row
    for ind in df_prod.index:
        
        year = df_prod.columns[i]
        energy = df_prod[year][ind]
        energy = int(energy)
        statecode = df_prod['StateCode'][ind]
        msncode = df_prod['MSN'][ind]

        # get source based on code
        source = df_code.loc[df_code['msnCode'] == msncode, 'sourceCode'].item()
        
        # add values as a list to the data list
        data.append([statecode, source, year, msncode, energy])
        
# create final production relation dataframe from lists and write to file
column_names = ['stateCode', 'sourceCode', 'year', 'msnCode', 'energyProduced']
final = pd.DataFrame(data, columns = column_names)
final.to_csv('./relations/production.csv', index=False)

# create production code relation from df_code dataframe and write to file
df_code = df_code[['msnCode', 'description', 'units']]
df_code.to_csv('./relations/production_codes.csv', index=False)

In [6]:
### EXTRACT THE CONSUMPTION RELATION DATA

df_cons = pd.read_excel('./sources/consumption_data.xlsx', sheet_name='Data')
df_code = pd.read_excel('./sources/consumption_codes.xlsx', sheet_name='Data')
df_subcode = pd.read_excel('./sources/consumption_codes.xlsx', sheet_name='temp')
df_state = pd.read_excel('./sources/states.xlsx', sheet_name='Data')

df_cons = df_cons.drop('Data_Status', axis=1)

valid_codes = df_code['Code'].tolist()
subcodes = df_subcode['subCode'].tolist()

df_subsources = df_cons.loc[df_cons['MSN'].isin(subcodes)]
df_cons = df_cons.loc[df_cons['MSN'].isin(valid_codes)]

RERCB = ['GERCB', 'SORCB', 'WDRCB']
RECCB = ['EMCCB', 'GECCB', 'HYCCB', 'SOCCB', 'WWCCB', 'WYCCB']
REICB = ['BDLCB', 'EMICB', 'EMLCB', 'GEICB', 'HYICB', 'SOICB', 'WWICB', 'WYICB']
REACB = ['BDACB', 'EMACB']
REEIB = ['BDACB', 'EMACB', 'SOEGB', 'WWEIB', 'WYEGB']

adddata = [RERCB, RECCB, REICB, REACB, REEIB]
datalabels = ['RERCB', 'RECCB', 'REICB', 'REACB', 'REEIB']

# create missing sector consumption rows for renewable energy
# using subcode metrics

state_codes = df_state['stateCode'].tolist()

for state in state_codes:
    
    for i in range(len(adddata)):
        new_row = df_subsources.loc[(df_subsources.State == state) & (df_subsources.MSN.isin(adddata[i]))].sum()
        new_row[0] = state
        new_row[1] = datalabels[i]
        series = pd.Series(new_row, index = df_cons.columns)
        df_cons = df_cons.append(series, ignore_index=True)

        
# will contain formatted data for output
data = []

# verify codes are filtered and formatted correctly
df_cons = df_cons.loc[df_cons['MSN'].isin(valid_codes)]    

# convert each year column of energy consumed into individual rows:
for i in range(2, 61):
    
    # get values from row
    for ind in df_cons.index:
        
        year = df_cons.columns[i]
        energy = df_cons[year][ind]
        energy = int(energy)
        statecode = df_cons['State'][ind]
        msncode = df_cons['MSN'][ind]

        # get source and sector info based on msn code
        source = df_code.loc[df_code['Code'] == msncode, 'SourceCode'].item()
        sector = df_code.loc[df_code['Code'] == msncode, 'SectorCode'].item()
        
        data.append([statecode, source, sector, year, msncode, energy])
        
        
# create final consumption relation dataframe from lists and write to file
column_names = ['stateCode', 'sourceCode', 'sectorCode', 'year', 'msnCode', 'energyProduced']
final = pd.DataFrame(data, columns = column_names)
final.to_csv('./relations/consumption.csv', index=False)

# create consumption code relation from df_code dataframe and write to file
df_code = df_code[['Code', 'Description', 'Units']]
df_code.to_csv('./relations/consumption_codes.csv', index=False)

In [69]:
### CREATE THE STATE RELATION DATA
df_state = pd.read_excel('./sources/states.xlsx', sheet_name='Data')
df_state.to_csv('./relations/states.csv', index=False)


In [70]:
### CREATE THE SOURCE RELATION DATA
df_state = pd.read_excel('./sources/sources.xlsx', sheet_name='Data')
df_state.to_csv('./relations/sources.csv', index=False)

In [71]:
### CREATE THE SECTOR RELATION DATA
df_state = pd.read_excel('./sources/sectors.xlsx', sheet_name='Data')
df_state.to_csv('./relations/sectors.csv', index=False)


In [72]:
### CREATE THE YEARS RELATION DATA
df_state = pd.read_excel('./sources/years.xlsx', sheet_name='Data')
df_state.to_csv('./relations/years.csv', index=False)


